In [ ]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from scipy.special import digamma
import csv

import pandas as pd
from sklearn.model_selection import train_test_split

# Configuración
NUM_FACTORS = 5
ALPHA = 0.8
BETA = 5
R = 4
MIN_RATING = 0
MAX_RATING = 10
NUM_ITERATIONS = 50

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')


In [1]:
!pip install scikit-surprise numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 48.1 MB/s eta 0:00:00
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505208 sha256=c220cecf072627d306ad4fade7d9ea26c8aa21d98d53313eb5ff5c4244430a14
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [ ]:
!pip list | grep numpy

numpy                              1.26.4


## SVDpp

In [3]:
import pandas as pd
from surprise import Dataset, Reader, SVDpp
from surprise.trainset import Trainset
import csv

# Cargar datos de entrenamiento
df_train = pd.read_csv("../data/train.csv")
reader = Reader(rating_scale=(0, 10))  # Asegúrate que coincide con tu escala
data = Dataset.load_from_df(df_train[['user', 'item', 'rating']], reader)
trainset: Trainset = data.build_full_trainset()

# Entrenar modelo SVD++
algo = SVDpp(n_factors=50, n_epochs=20, reg_all=0.02, lr_all=0.007, verbose=True)
algo.fit(trainset)

# Cargar datos de test
df_test = pd.read_csv("../data/test.csv")

# Mapas para IDs originales
user_inner_id = trainset._raw2inner_id_users
item_inner_id = trainset._raw2inner_id_items

# Predecir y guardar
output_rows = []
for _, row in df_test.iterrows():
    test_id = row['ID']
    user = row['user']
    item = row['item']

    # Surprise maneja automáticamente nuevos usuarios/items como unknown
    pred = algo.predict(str(user), str(item)).est
    pred = round(max(0, min(10, pred)), 3)

    output_rows.append((test_id, pred))

# Guardar predicciones
with open("predicciones_SVDpp_50_20_0.02.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["ID", "rating"])
    writer.writerows(output_rows)

print("Archivo generado correctamente.")


 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
Archivo generado correctamente.


In [7]:
import pandas as pd
from surprise import Dataset, Reader, SVDpp
from surprise.model_selection import GridSearchCV
import csv

# Cargar datos
df_train = pd.read_csv("../data/train.csv")

# Convertir a formato surprise
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_train[['user', 'item', 'rating']], reader)

# Definir rejilla de hiperparámetros
param_grid = {
    'n_factors': [20, 50, 100],
    'n_epochs': [10, 20],
    'reg_all': [0.02, 0.05, 0.1],
    'lr_all': [0.005, 0.007, 0.01]
}

# Búsqueda con validación cruzada
gs = GridSearchCV(SVDpp, param_grid, measures=['mae'], cv=3, joblib_verbose=1, n_jobs=10)
gs.fit(data)

# Mostrar mejores hiperparámetros
print("Mejores hiperparámetros:")
print(gs.best_params['mae'])
print(f"Mejor MAE: {gs.best_score['mae']:.4f}")

# Entrenar modelo final con mejores parámetros
best_algo = gs.best_estimator['mae']
trainset = data.build_full_trainset()
best_algo.fit(trainset)

# Cargar test
df_test = pd.read_csv("../data/test.csv")

# Predecir y guardar resultados
output_rows = []
for _, row in df_test.iterrows():
    test_id = row['ID']
    user = str(row['user'])
    item = str(row['item'])

    pred = best_algo.predict(user, item).est
    pred = round(max(0, min(10, pred)), 3)

    output_rows.append((test_id, pred))

# Guardar CSV
filename = "predicciones_SVDpp_gridsearch.csv"
with open(filename, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["ID", "rating"])
    writer.writerows(output_rows)

print(f"Archivo '{filename}' generado correctamente.")

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import NMF
from surprise.model_selection import GridSearchCV
import csv

# Cargar datos
df_train = pd.read_csv("train.csv")

# Convertir a formato surprise
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_train[['user', 'item', 'rating']], reader)

# Definir rejilla de hiperparámetros
param_grid = {
    'n_factors': [20, 50, 100],  # Número de factores latentes
    'n_epochs': [10, 20],        # Número de épocas
    'reg_pu': [0.02, 0.05, 0.1], # Regularización de usuarios
    'reg_qi': [0.02, 0.05, 0.1], # Regularización de ítems
    'biased': [True, False]      # Usar sesgo (media global)
}

# Búsqueda de hiperparámetros con GridSearchCV
gs = GridSearchCV(NMF, param_grid, measures=['mae'], cv=3, joblib_verbose=1, n_jobs=-1)
gs.fit(data)

# Mostrar mejores hiperparámetros
print("Mejores hiperparámetros:")
print(gs.best_params['mae'])
print(f"Mejor MAE: {gs.best_score['mae']:.4f}")

# Entrenar modelo con mejores hiperparámetros
best_algo = gs.best_estimator['mae']
trainset = data.build_full_trainset()
best_algo.fit(trainset)

# Cargar datos de test
df_test = pd.read_csv("test.csv")

# Generar predicciones
output_rows = []
for _, row in df_test.iterrows():
    test_id = row['ID']
    user = str(row['user'])
    item = str(row['item'])

    # Obtener predicción
    pred = best_algo.predict(user, item).est
    pred = round(max(0, min(10, pred)), 3)

    output_rows.append((test_id, pred))

# Guardar predicciones en CSV
filename = f"predicciones_NMF_gridsearch.csv"
with open(filename, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["ID", "rating"])
    writer.writerows(output_rows)

print(f"Archivo '{filename}' generado correctamente.")


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 38.7min
[Parallel(n_jobs=-1)]: Done 324 out of 324 | elapsed: 95.3min finished


Mejores hiperparámetros:
{'n_factors': 20, 'n_epochs': 20, 'reg_pu': 0.1, 'reg_qi': 0.1, 'biased': True}
Mejor MAE: 1.4635
Archivo 'predicciones_NMF_gridsearch.csv' generado correctamente.


In [2]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate
from ray import tune
import csv
import ray

2025-03-29 09:45:47,533	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-29 09:45:47,609	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise import accuracy
import csv
import matplotlib.pyplot as plt

# Cargar datos
df_train = pd.read_csv("train.csv")

# Convertir a formato surprise
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_train[['user', 'item', 'rating']], reader)

# Dividir en entrenamiento y validación (80/20)
trainset, valset = train_test_split(data, test_size=0.2, random_state=42)

# Definir rejilla de hiperparámetros
param_grid = {
    'k': [20, 25, 30, 35],                      # Número de vecinos
    'min_k': [5, 10],                           # Número mínimo de vecinos
    'sim_options': {
        'name': ['pearson_baseline', 'cosine'],   # Similaridad
        'user_based': [True, False]             # Comparar usuarios o ítems
    }
}

# Búsqueda de hiperparámetros con GridSearchCV
print("Iniciando búsqueda de hiperparámetros...")
gs = GridSearchCV(KNNBaseline, param_grid, measures=['mae'], cv=3, joblib_verbose=1, n_jobs=-1)
gs.fit(data)

# Mostrar mejores hiperparámetros
print("\nMejores hiperparámetros encontrados:")
print(gs.best_params['mae'])
print(f"Mejor MAE: {gs.best_score['mae']:.4f}")

# Entrenar modelo con mejores parámetros
print("\nEntrenando el modelo final con los mejores parámetros...")
best_algo = gs.best_estimator['mae']
trainset = data.build_full_trainset()
best_algo.fit(trainset)

# Cargar datos de test
df_test = pd.read_csv("test.csv")

# Guardar MAE por iteración
mae_per_iteration = []

# Generar predicciones y calcular MAE
print("\nGenerando predicciones y calculando MAE...")
output_rows = []
for _, row in df_test.iterrows():
    test_id = row['ID']
    user = str(row['user'])
    item = str(row['item'])

    # Obtener predicción
    pred = best_algo.predict(user, item)
    est = round(max(0, min(10, pred.est)), 3)

    # Calcular MAE acumulado
    mae_per_iteration.append(abs(pred.r_ui - est) if pred.r_ui is not None else 0)

    output_rows.append((test_id, est))

    # Mostrar progreso cada 1000 predicciones
    if len(output_rows) % 1000 == 0:
        current_mae = sum(mae_per_iteration) / len(mae_per_iteration)
        print(f"Predicción #{len(output_rows)} - MAE acumulado: {current_mae:.4f}")

# Guardar predicciones en CSV
filename = "predicciones_KNNBaseline.csv"
with open(filename, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["ID", "rating"])
    writer.writerows(output_rows)

print(f"\nArchivo '{filename}' generado correctamente.")

# Calcular MAE final
final_mae = sum(mae_per_iteration) / len(mae_per_iteration)
print(f"\nMAE final en test: {final_mae:.4f}")

Iniciando búsqueda de hiperparámetros...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}